分析をするための前準備

In [ ]:
#import modules

#dataframe
import pandas as pd

#number processing
import numpy as np

#standard scaling
from sklearn.preprocessing import StandardScaler

#plotting(このファイルでは不使用)
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt

#heatmap(このファイルでは不使用)
import seaborn as sns

#math
import math

In [ ]:
#make the list

#mouse
mouse_list = ["ID138464-1","ID138464-2","ID138464-3","ID138464-4","ID138464-5","ID138464-6"]
#all phase 
phaseall_list=["HC1","B1","B2","HC2","A1","A2","A31","A32","A41","A42","HC3","A5","B3","A6","A7","HC4","B4"]
#phase targeted for anlysis
phase_list=["A1","A2","A31","A32","A41","A42","A5","A6","A7"]

In [ ]:
#前処理

#mouseごとに処理をする
for m in range(0,len(mouse_list)):

    mouse = mouse_list[m]
    
    #分析用にフェーズごとのcsvファイルを作成

    #神経データをデータフレームにインポートする
    inputf='%s/python/%s_Longitudinal_Traces.csv' %(mouse,mouse)
    dff=pd.read_csv(inputf,header=0)
    
    #Acceptのみを抽出したデータフレームの作成
    dff0=pd.DataFrame()
    dff0=dff.iloc[0,:]
    Accept=dff0.index[dff0==' accepted']
    dfa=pd.DataFrame()
    dfa['Time2']=dff.iloc[1:,0]
    for accept_c in range (0,len(Accept)):
        dfa[Accept[accept_c]]=dff[Accept[accept_c]].iloc[1:]
        dfa=dfa.astype('float')

    #Acceptcellのfirst cellとlast cellを抽出(後の処理のため)
    cell_f=dfa.columns[1]
    cell_l=dfa.columns[dfa.shape[1]-1]

    #レコード間の時間差からphaseを割り当てる
    dfa['dt']=dfa['Time2'].diff()
    dfa['dt'].iloc[0]=0.05
    dfa['phase']=-1
    phase=0
    for i in range(0,dfa.shape[0]):    
        if(dfa['dt'].iloc[i]>10):        
            phase=phase+1
        dfa.iloc[i,dfa.columns.get_loc('phase')]=phaseall_list[phase]

    #割り当てたphaseごとにcsvを生成(XYdataがphaseごとに生成されているため)
    for phase in range(0,len(phaseall_list)):    
        outputf='%s/python/%s_%s_Long_Traces.csv' %(mouse,mouse,phaseall_list[phase])    
        df_phase=dfa[dfa['phase']==phaseall_list[phase]]        
        df_phase.to_csv(outputf,index=False)

    #XYデータにヘッダー情報がたくさん入っている、かつcsvなのでヘッダー情報をカラム情報以外取り除き、csvに出力する
    phasepre_list=["B1","B2","A1","A2","A3","A4","A5","B3","A6","A7","B4"]
    for phase_pre in range(0,len(phasepre_list)):
        if (phase_pre==phasepre_list.index("A3")) or (phase_pre==phasepre_list.index("A4")):
            break
        inputXY='%s/python/%s_XYdata/%s_%s_%s_XY.txt' %(mouse,mouse,mouse,phase_pre+1,phasepre_list[phase_pre])
        outputXY='%s/python/%s_XYdata/%s_%s_XY.csv' %(mouse,mouse,mouse,phasepre_list[phase_pre])
        dfxyp=pd.read_table(inputXY)
        h=dfxyp[dfxyp['TimeFZ4']=='Slice'].index[0]
        dfxy=pd.read_table(inputXY,header=h+1)
        dfxy.to_csv(outputXY,index=False)

    #XYデータがA3,A4を前半後半に分割していないので分割してからcsvに
    phasetransb_list=["A3","A4"]
    phasetransa_list=["A31","A32","A41","A42"]
    for phase_b in range(0,2):
        inputXY='%s/python/%s_XYdata/%s_%s_%s_XY.txt' %(mouse,mouse,mouse,phase_b+5,phasetransb_list[phase_b])
        for phase_a in range(0, 2):
            outputXY='%s/python/%s_XYdata/%s_%s_XY.csv' %(mouse,mouse,mouse,phasetransa_list[2*phase_b+phase_a])
            dfxyp=pd.read_table(inputXY)
            h=dfxyp[dfxyp['TimeFZ4']=='Slice'].index[0]
            dfxy=pd.read_table(inputXY,header=h+1)
            dfxy.iloc[3000*phase_a+0:3000*phase_a+621,:].to_csv(outputXY,index=False)

    #後ほど使う全phase合わせた用のデータフレーム
    dfmeandiff_all=pd.DataFrame()

    #phaseごとに分析をする
    for ph in range(0,len(phase_list)):
        phase=phase_list[ph]
        inputXY='%s/python/%s_XYdata/%s_%s_XY.csv' %(mouse,mouse,mouse,phase)
        dfx = pd.read_csv(inputXY,header=0)
        inputT='%s/python/%s_%s_Long_Traces.csv' %(mouse,mouse,phase)
        dfl=pd.read_csv(inputT,header=0)
"""
        #FZ-nonFZ状態の入れ替わりからsessionを付与(現在は不使用)
        dfx["session"]=0
        ses=dfx.columns.get_loc('session')
        for i in range(1,dfx.shape[0]):
            if (dfx.iloc[i].loc["FZ"]!=dfx.iloc[i-1].loc["FZ"]):
                dfx.iloc[i,ses]= int(dfx.iloc[i-1,ses])+1
            else:
                dfx.iloc[i,ses]= int(dfx.iloc[i-1,ses])

        #後ほどFZの短いセッション結合を行うための準備(現在は不使用)
        dfx['FZ_c0']=dfx['FZ']
        dfx['ses_c0']=dfx['session']
        ses_cnt=0
        FZ_col=dfx.columns.get_loc('FZ_c0')
        session_col=dfx.columns.get_loc('ses_c0')
        ses_c='ses_c%s'%(ses_cnt)
        mconseq='mconseq_%s'%(ses_cnt)
        dfx[mconseq]=0
        mconseq_col=dfx.columns.get_loc('mconseq_0')
        Slice_col=dfx.columns.get_loc('Slice')

        #mconseq(マイナス方向に伸ばしたconseqFZ)を付与(現在は不使用)
        for i in range (0,dfx.shape[0]):
            ses_no = dfx.iloc[i,session_col]
            if dfx.iloc[i,FZ_col] == 0:
                if ses_no == dfx.iloc[dfx.shape[0]-1].loc[ses_c]:
                    dfx.iloc[i,mconseq_col]=-100
                    break;
                df_ns=dfx[dfx[ses_c]==ses_no+1]
            else:
                df_ns=dfx[dfx[ses_c]==ses_no]
            df_ns1=df_ns.iloc[0]
            dfx.iloc[i,mconseq_col]=int(dfx.iloc[i,Slice_col]-df_ns1.iloc[Slice_col]+1)

        #validラベルを付与(現在は不使用)
        valid_fz='valid_%s'%(ses_cnt)
        dfx[valid_fz]=0
        valid_col=dfx.columns.get_loc(valid_fz)
        for i in range(0,dfx.shape[0]):
            ses_no_v = dfx.iloc[i,session_col]
            if dfx.iloc[i,FZ_col] == 0:
                if ses_no_v == dfx.iloc[dfx.shape[0]-1].loc[ses_c]:
                    break;
                df_ns_v=dfx[dfx[ses_c]==ses_no_v+1]
            else:
                df_ns_v=dfx[dfx[ses_c]==ses_no_v]
            df_ns1_v=df_ns_v.iloc[df_ns_v.shape[0]-1]
            if df_ns1_v[mconseq_col]<9:
                dfx.iloc[i,valid_col]=0
            else:
                dfx.iloc[i,valid_col]=1

        #短いFZセッションの結合→mconseqを付与→validを付与(現在は不使用)
        for ses_cnt in range(1,10):
            FZ_c='FZ_c%s'%(ses_cnt)
            FZ_cm='FZ_c%s'%(ses_cnt-1)
            ses_c='ses_c%s'%(ses_cnt)
            ses_cm='ses_c%s'%(ses_cnt-1)
            dfx[FZ_c]=dfx[FZ_cm]
            dfx[ses_c]=0
            fzc=dfx.columns.get_loc(FZ_c)
            cnt=0
            sup=0
            r=20
            for i in range(1,int(dfx.iloc[dfx.shape[0]-1].loc[ses_cm])+1):
                fz=dfx[dfx[ses_cm]==i].iloc[0,fzc]
                nfz=int((1-fz)*(1-fz))
                slice_b=dfx[dfx[ses_cm]==i].iloc[0].loc['Slice']
                slice_a=dfx[dfx[ses_cm]==i].iloc[dfx[dfx[ses_cm]==i].shape[0]-1].loc['Slice']
                Queryp='%d<=Slice<%d'%(int(slice_b),int(slice_a+r))
                Querym='%d<=Slice<%d'%(int(slice_b),int(slice_a+r))
                dfx_sp=dfx.query(Queryp)
                dfx_sm=dfx.query(Querym)
                if(i<int(dfx.iloc[dfx.shape[0]-1].loc[ses_cm])):
                    pp=1.0*dfx_sp[FZ_cm].value_counts()[fz]/(dfx_sp.shape[0]*1.0)
                    pm=1.0*dfx_sm[FZ_cm].value_counts()[fz]/(dfx_sm.shape[0]*1.0)
                else:
                    p=1
                if((dfx[dfx[ses_cm]==i].shape[0]<10) and (pp<0.350 or pm<0.350) and  (fz==0)):
                    dfx.loc[dfx[ses_cm]==i,ses_c]=cnt
                    dfx.loc[dfx[ses_cm]==i,FZ_c]=nfz
                    sup=1
                else:
                    cnt+=1-sup
                    dfx.loc[dfx[ses_cm]==i,ses_c]=cnt
                    dfx.loc[dfx[ses_cm]==i,FZ_c]=fz
                    sup=0
            mconseq='mconseq_%s'%(ses_cnt)
            dfx[mconseq]=-10
            FZ_col=dfx.columns.get_loc(FZ_c)
            session_col=dfx.columns.get_loc(ses_c)
            mconseq_col=dfx.columns.get_loc(mconseq)
            Slice_col=dfx.columns.get_loc('Slice')
            for i in range (0,dfx.shape[0]):
                ses_no = dfx.iloc[i,session_col]
                if dfx.iloc[i,FZ_col] == 0:
                    if ses_no == dfx.iloc[dfx.shape[0]-1].loc[ses_c]:
                        break;
                    df_ns=dfx[dfx[ses_c]==ses_no+1]
                else:
                    df_ns=dfx[dfx[ses_c]==ses_no]
                df_ns1=df_ns.iloc[0]
                dfx.iloc[i,mconseq_col]=int(dfx.iloc[i,Slice_col]-df_ns1.iloc[Slice_col]+1)
            valid_fz='valid_%s'%(ses_cnt)
            dfx[valid_fz]=0
            valid_col=dfx.columns.get_loc(valid_fz)
            for i in range(0,dfx.shape[0]):
                ses_no_v = dfx.iloc[i,session_col]
                if dfx.iloc[i,FZ_col] == 0:
                    if ses_no_v == dfx.iloc[dfx.shape[0]-1].loc[ses_c]:
                        break;
                    df_ns_v=dfx[dfx[ses_c]==ses_no_v+1]
                else:
                    df_ns_v=dfx[dfx[ses_c]==ses_no_v]
                df_ns1_v=df_ns_v.iloc[df_ns_v.shape[0]-1]
                if df_ns1_v[mconseq_col]<9:
                    dfx.iloc[i,valid_col]=0
                else:
                    dfx.iloc[i,valid_col]=1
            if(dfx.iloc[dfx.shape[0]-1].loc[ses_cm]==dfx.iloc[dfx.shape[0]-1].loc[ses_c]):
                break;
"""
        #ここで神経データと行動データを結合 して、csvファイルを作成     
        dfx_m=pd.DataFrame(columns=dfx.columns,index=np.arange(0,int(dfx.iloc[dfx.shape[0]-1].loc['Slice']-1)*10))
        for i in range(2,int(dfx.shape[0])):
            for j in range(0,10):
                for k in range(0,len(dfx_m.columns)):
                    dfx_m.iloc[(i-2)*10+j].loc[dfx_m.columns[k]]=dfx.iloc[i].loc[dfx_m.columns[k]]
        df=pd.concat([dfx_m, dfl], axis=1)
        df=df.dropna(how='any')
        outputf='%s/python/%s_%s_con.csv' %(mouse,mouse,phase)          
        df.to_csv(outputf,index=False)

        #Sliceごとに平均をとったdfmeanの作成
        Index=np.arange(int(df.iloc[0].loc['Slice']),int(df.iloc[df.shape[0]-1].loc['Slice']+1),1)
        df_mean=pd.DataFrame(index=Index,columns=[])
        for i in range(0,len(df.columns)):
            df_mean[df.columns[i]]=-100
        for i_n in range(int(df.iloc[0].loc['Slice']),int(df.iloc[df.shape[0]-1].loc['Slice']+1)):
            i = i_n - int(df.iloc[0].loc['Slice'])
            df_mean.iloc[i].loc['Slice']=i_n
            dfs = df[df['Slice']==i_n]
            for k in range(0,df.columns.get_loc('Time2')+1):
                df_mean.iloc[i,k]=dfs.iloc[0,k]
            for j in range(dfs.columns.get_loc(cell_f),dfs.columns.get_loc(cell_l)+1):
                df_mean.iloc[i,j]=dfs[dfs.columns[j]].mean()
            #現在不使用のcolumn関係
            #for l in range(dfs.columns.get_loc('session'),dfs.columns.get_loc(ses_c)+1):
                #df_mean.iloc[i,l]=int(dfs.iloc[0,l])
        dfmean=df_mean
        outputf='%s/python/%s_%s_mean.csv' %(mouse,mouse,phase)          
        dfmean.to_csv(outputf,index=False)
        
        #dfmeanのdiff(変化分)を作成
        dfmeandiff=df_mean.diff()
        dfmeandiff.loc[:,:'Time2']=df_mean.loc[:,:'Time2']
        dfmeandiff=dfmean.dropna(how='any')
        dfmeandiff_std=pd.DataFrame(columns=dfmeandiff.columns,index=dfmeandiff.index)
        for k in range(dfmeandiff_std.columns.get_loc('Slice'),dfmeandiff_std.columns.get_loc(cell_f)):
            dfmeandiff_std[dfmeandiff_std.columns[k]]=dfmeandiff[dfmeandiff_std.columns[k]]
        for i in range(dfmeandiff_std.columns.get_loc(cell_f),dfmeandiff_std.columns.get_loc(cell_l)+1):
            for j in range(0,dfmeandiff_std.shape[0]):
                dfmeandiff_std.iloc[j,i]=((dfmeandiff.iloc[j,i]-dfmeandiff[dfmeandiff_std.columns[i]].mean())/dfmeandiff[dfmeandiff_std.columns[i]].std())
        dfmeandiff_all=pd.concat([dfmeandiff_all,dfmeandiff_std])
        outputf='%s/python/%s_%s_meandiff.csv' %(mouse,mouse,phase)          
        dfmeandiff_std.to_csv(outputf,index=False)
        

for m in range(0,len(mouse_list)):
    mouse=mouse_list[m]
    
    #神経データをデータフレームにインポートする
    inputf='%s/python/%s_Longitudinal_Traces.csv' %(mouse,mouse)
    dff=pd.read_csv(inputf,header=0)
    dff0=pd.DataFrame()

    #Acceptのみを抽出したデータフレームの作成
    dff0=dff.iloc[0,:]
    Accept=dff0.index[dff0==' accepted']
    dfa=pd.DataFrame()
    dfa['Time2']=dff.iloc[1:,0]
    for accept_c in range (0,len(Accept)):
        dfa[Accept[accept_c]]=dff[Accept[accept_c]].iloc[1:]
        dfa=dfa.astype('float')

    #Acceptcellのfirst cellとlast cellを抽出
    cell_f=dfa.columns[1]
    cell_l=dfa.columns[dfa.shape[1]-1]
    
    dfmean_all=pd.DataFrame()
    dfmeanstd_all=pd.DataFrame()
    
    for ph in range(0,len(phase_list)):
        
        phase=phase_list[ph]
        
        inputf='%s/python/%s_%s_mean.csv' %(mouse,mouse,phase)
        
        dfmean=pd.read_csv(inputf,header=0)
        
        dfmean_all=pd.concat([dfmean_all,dfmean])
        
        #sliceiniの情報をstd化
        dfmean_std=pd.DataFrame(columns=dfmean.columns,index=dfmean.index)
        for k in range(dfmean_std.columns.get_loc('Slice'),dfmean_std.columns.get_loc(cell_f)):
            dfmean_std[dfmean_std.columns[k]]=dfmean[dfmean_std.columns[k]]
        for i in range(dfmean_std.columns.get_loc(cell_f),dfmean_std.columns.get_loc(cell_l)+1):
            for j in range(0,dfmean_std.shape[0]):
                dfmean_std.iloc[j,i]=((dfmean.iloc[j,i]-dfmean[dfmean_std.columns[i]].mean())/dfmean[dfmean_std.columns[i]].std())
        dfmeanstd_all=pd.concat([dfmeanstd_all,dfmean_std])
        outputf='%s/python/%s_%s_meanstd.csv' %(mouse,mouse,phase)          
        dfmean_std.to_csv(outputf,index=False)
    outputf='%s/python/%s_meanall.csv'%(mouse,mouse)
    dfmean_all.to_csv(outputf,index=False)
    outputf='%s/python/%s_meanstdall.csv' %(mouse,mouse)          
    dfmeanstd_all.to_csv(outputf,index=False)
    
"""
    #フェーズごとのdiffのデータフレームを、一つにまとめたデータフレームを作る(現在は不使用)
    
    dfmeandiff_all=pd.DataFrame()
    
    for ph in range(0,len(phase_list)):
        
        phase=phase_list[ph]
        
        inputf='%s/python/%s_%s_meandiff.csv' %(mouse,mouse,phase)
        
        dfmeandiffph=pd.read_csv(inputf,header=0)
        
        dfmeandiff_all=pd.concat([dfmeandiff_all,dfmeandiffph])
        
        outputf='%s/python/%s_meandiffall.csv' %(mouse,mouse)
        
    dfmeandiff_all.to_csv(outputf, index=False)

"""